In [1]:
import pandas as pd
import seaborn as sns
import rho_plus as rp

theme, cs = rp.mpl_setup(False)

In [2]:
import crystal_toolkit.components as ctc
import dash
from dash import html
from pymatgen.core import Structure


def quick_view(struct: Structure, **kwargs):
    app = dash.Dash()

    component = ctc.StructureMoleculeComponent(struct, **kwargs)
    ctc.register_crystal_toolkit(app, layout=html.Div([html.H2(struct.composition.to_unicode_string()), component.layout()]))

    app.run(port=8051)

No module named 'phonopy'


In [3]:
df = pd.read_pickle('merged_test_data3.pkl').set_index('material_id')
df

,formula_pretty,nsites,spacegroup,nelements,elements_list,CrystalSystem,category,nontrivial_coordinates,struct
material_id,,,,,,,,,
mp-557997,CaSeO3,20,14,3,Ca O Se,Monoclinic,polymorph_ternary,14,"[[3.34824742 5.7240056 5.93286188] Ca, [0.887..."
mp-13171,YMgCu,9,189,3,Cu Mg Y,Hexagonal,polymorph_ternary,1,"[[-2.18135055 3.7782094 0. ] Y, [ 4...."
mp-7550,CeNbO4,12,15,3,Ce Nb O,Monoclinic,polymorph_ternary,7,"[[4.11571443 0.6447703 3.28952249] Ce, [-0.53..."
mp-23550,KBrF4,12,140,3,Br F K,Tetragonal,polymorph_ternary,3,"[[1.05567769 1.34280835 2.17287608] K, [ 3.165..."
mp-5126,ZnSO4,24,62,3,O S Zn,Orthorhombic,polymorph_ternary,8,"[[2.37637601 3.33392793 4.30100142] Zn, [0. ..."
...,...,...,...,...,...,...,...,...,...
mp-1106064,Ho4Ga2Ni,17,229,3,Ga Ho Ni,Cubic,template-based_ternary,1,"[[ 2.11805495 2.11805495 -2.11805495] Ho, [ 2..."
mp-1105955,Er3Cu3Sb4,20,220,3,Cu Er Sb,Cubic,template-based_ternary,5,"[[-2.3728156 3.55922339 4.74563119] Er, [2...."
mp-1105893,La3Cu3Bi4,20,220,3,Bi Cu La,Cubic,template-based_ternary,5,"[[-2.52068287 3.7810243 5.04136574] La, [2...."


In [31]:
df.query('CrystalSystem == "Cubic"').iloc[[15]]

,formula_pretty,nsites,spacegroup,nelements,elements_list,CrystalSystem,category,nontrivial_coordinates,struct
material_id,,,,,,,,,
mp-20389,Na2CdPb,4,216,3,Cd Na Pb,Cubic,ternary_easy,0,"[[1.50230795 1.06229224 2.60207388] Na, [3.004..."


In [32]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

mp_id = 'mp-20389'
struct = df.loc[mp_id, 'struct']

sga = SpacegroupAnalyzer(struct)
symm = sga.get_conventional_standard_structure()
quick_view(symm)

In [33]:
symm.num_sites / struct.num_sites

4.0

In [34]:
from pymatgen.core import Lattice

lat_params = {
    'cubic': 'a',
    'hexagonal': ('a', 'c'),
    'tetragonal': ('a', 'c'),
    'orthorhombic': ('a', 'b', 'c'),
    'triclinic': ('a', 'b', 'c', 'alpha', 'beta', 'gamma'),
    'monoclinic': ('a', 'b', 'c', 'beta'),    
}

lat_funcs = {k: getattr(Lattice, k) if k != 'triclinic' else Lattice.from_parameters for k in lat_params.keys()}

In [36]:
import numpy as np
from pyxtal import Group
sg = Group(df.loc[mp_id, 'spacegroup'])
n_els = np.array(list(struct.composition.values()))

combs = []
scales = []
for scale in [int(symm.num_sites / struct.num_sites)]:
    scale_combs, has_freedom, inds = sg.list_wyckoff_combinations(n_els * scale)
    combs.extend(scale_combs)
    scales.extend([scale] * len(scale_combs))

len(combs)

12

In [37]:
from ray import tune
import numpy as np

lattice = {param: tune.uniform(2, 30) if param in 'abc' else tune.uniform(20, 160) for param in lat_params[sg.lattice_type]}

def coord_space(config):
    comb = combs[config['wp_i']]    
    symbs = sum(comb, start=[])
    dof = sg.get_site_dof(symbs)
    return np.random.uniform(size=int(sum(dof)))

wp_i = {'wp_i': tune.randint(0, len(combs))}
coords = {'wp_xyz': tune.sample_from(coord_space)}

config = {}
for d in (lattice, wp_i, coords):
    config.update(d)

In [38]:
from pymatgen.core import Structure, Lattice
from pyxtal import Wyckoff_position
from baysic.feature_space import FeatureSpace
import numpy as np

def make_struct(config):
    lat_vals = [config[param] for param in lattice]
    lat = lat_funcs[sg.lattice_type](*lat_vals)
    fs = FeatureSpace(sg.number)    
    flat_combs = []
    flat_elems = []
    comb = combs[config['wp_i']]
    for el_group, el in zip(comb, struct.elements):
        flat_combs.extend(el_group)
        flat_elems.extend([el] * len(el_group))

    dofs = fs.sg.get_site_dof(flat_combs)
    coord_i = 0
    coords = []
    elems = []
    for letter, elem, dof in zip(flat_combs, flat_elems, dofs):
        dof = int(dof)
        wp = Wyckoff_position.from_group_and_letter(sg.number, letter)
        positions = wp.get_all_positions(fs.from_free_transformed_xyz(config['wp_xyz'][coord_i:coord_i+dof], wp))
        coords.append(positions)
        elems.extend([elem] * len(positions))
        coord_i += dof

    coords = np.vstack(coords)
    return Structure(lat, elems, coords)

In [39]:
from __future__ import annotations

import warnings

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from pymatgen.core import Lattice, Structure
from pymatgen.io.ase import AseAtomsAdaptor

import matgl
from matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer


pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")
relaxer = Relaxer(potential=pot)

In [40]:
import itertools
from scipy.spatial.distance import pdist, squareform

MIN_DIST_RATIO = 0.4
VACUUM_SIZE = 7.0
def is_structure_valid(struct: Structure) -> bool:
    """Tests structure validity."""
    struct.make_supercell([2, 2, 2], to_unit_cell=False)
    
    # distance threshold
    dists = squareform(pdist(struct.cart_coords))
    for i, j in itertools.combinations(range(struct.num_sites), 2):
        if dists[i, j] < sum([struct.sites[c].specie.atomic_radius for c in (i, j)]) * MIN_DIST_RATIO:
            return False
        

    def get_foot(p, a, b):
        p = np.array(p)
        a = np.array(a)
        b = np.array(b)
        ap = p - a
        ab = b - a
        result = a + np.dot(ap, ab) / np.dot(ab, ab) * ab
        return result

    def get_distance(a, b):
        return np.sqrt(np.sum(np.square(b - a)))


    line_a_points = [[0, 0, 0], ]
    line_b_points = [[0, 0, 1], [0, 1, 0], [1, 0, 0],
                        [0, 1, 1], [1, 0, 1], [1, 1, 0], [0, 1, -1], [1, 0, -1], [1, -1, 0],
                        [1, 1, 1], [1, 1, -1], [1, -1, 1], [-1, 1, 1]]
    for a in line_a_points:
        for b in line_b_points:
            foot_points = []
            for p in struct.frac_coords:
                f_p = get_foot(p, a, b)
                foot_points.append(f_p)
            foot_points = sorted(foot_points, key=lambda x: [x[0], x[1], x[2]])

            # 转为笛卡尔坐标
            foot_points = np.asarray(np.mat(foot_points) * np.mat(struct.lattice.matrix))
            for fp_i in range(0, len(foot_points) - 1):
                fp_distance = get_distance(foot_points[fp_i + 1], foot_points[fp_i])
                if fp_distance > VACUUM_SIZE:
                    return False
        
    return True

def relaxed_energy(struct: Structure, long: bool = False) -> (Structure, float):
    if long:
        params = dict(fmax=0.01, steps=300)    
    else:
        params = dict(fmax=0.02, steps=5)
        
    relax_results = relaxer.relax(struct, **params)
    # extract results
    final_structure = relax_results["final_structure"]
    final_energy = relax_results["trajectory"].energies[-1]
    # print out the final relaxed structure and energy

    return (final_structure, final_energy)

def objective(config) -> float:
    struct = make_struct(config)
    if not is_structure_valid(struct):
        return 100
    else:
        return relaxed_energy(struct)[1] / struct.composition.num_atoms

In [41]:
tuner = tune.Tuner(
    objective, 
    param_space=config,
    tune_config = tune.TuneConfig(
        mode='min',  
        num_samples=30,
        max_concurrent_trials=5
    )
)
res = tuner.fit().get_best_result()
res.config

2023-09-25 20:40:17,366	ERROR tune_controller.py:1502 -- Trial task failed for trial objective_41fbc_00004
Traceback (most recent call last):
  File "/home/nicholas/anaconda3/envs/baysic/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/nicholas/anaconda3/envs/baysic/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/nicholas/anaconda3/envs/baysic/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/nicholas/anaconda3/envs/baysic/lib/python3.11/site-packages/ray/_private/worker.py", line 2547, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(IndexError): ray::ImplicitFunc.train() (pid=741449, ip=192.168.0.88, acto

RuntimeError: No best trial found for the given metric: _metric. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [45]:
sg

-- Spacegroup --# 216 (F-43m)--
96i	site symm: 1
48h	site symm: . . m
24g	site symm: 2 . mm
24f	site symm: 2 . mm
16e	site symm: . 3 m
4d	site symm: -4 33 mm
4c	site symm: -4 33 mm
4b	site symm: -4 33 mm
4a	site symm: -4 3 m

In [42]:
final_struct = make_struct(res.config)
final_relaxed, final_energy = relaxed_energy(final_struct, long=True)
print(final_energy / final_struct.composition.num_atoms)
quick_view(final_relaxed)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [44]:
fs = 

{'a': 3.964484424677978, 'wp_i': 0, 'wp_xyz': array([], dtype=float64)}

In [ ]:

sga.find_primitive(struct).get_space_group_info()

In [ ]:
dir(struct)




In [ ]:
quick_view(final_struct)

In [ ]:
print(symm.matches(final_relaxed))
quick_view(symm)